In [ ]:
#| default_exp agents

# Imports

In [ ]:
#| export

# From lib
import projective_simulation.methods.preprocessors as preprocessors
import projective_simulation.ECMs as ECMs

# From others
from abc import ABC, abstractmethod

# Abstract Agent

In [ ]:
#| export
class Abstract_Agent(ABC):
    """
    A minimal class every agent should fullfill, every agent should be Derived from this class
    """

    def __init__(self, 
                 ECM = None, #The ECM Object to use
                 percept_processor = None, #An optional object for transforming observations prior to passing to ECM as a percept. Must have method "preprocess" 
                 action_processor = None #An optional object for transforming actions prior to passing to Environment as an actuator state. Must have method "postprocess" 
                ):
        self.ECM = ECM
        assert ECM is not None

        self.percept_processor = percept_processor
        if percept_processor is not None:
            assert hasattr(percept_processor, "preproccess")

        self.action_processor = action_processor
        if action_processor is not None:
            assert hassattr(action_processor, "postprocess")

    @abstractmethod
    def update(self, 
               reward, #used to reinforce agent parameters based on desirable or aversive states
               observation #data object passed from the environment/sensors
              ):

        raise NotImplementedError
    
    @abstractmethod
    def get_action(self, 
                   percept #Data object to be processed by ECM. Likely a list np.array
                  ):

        raise NotImplementedError

# Basic Agent

In [ ]:
#| export
class Basic_PSAgent(Abstract_Agent):
    def __init__(self, 
                 ECM = None, #if an ECM object is not given, a number of actions must be given with whi
                 num_actions = None, # The number of available actions. If an ECM is not given, should be int
                 glow: float = 0.1, # The glow (or eta) parameter. Won't be used if ECM is given
                 damp: float = 0., # The damping (or gamma) parameter. Won't be used if ECM is given
                 softmax: float = 0.1, # The softmax (or beta) parameter. Won't be used if ECM is given
                 percept_processor = preprocessors.get_percept, 
                 action_processor = None
                ):
        """
        Simple, projective simulation (PS) agent that uses a two-layer ECM. Percepts are added to the ECM as new obsevations are encountered
                      
        NOTE: This simple version misses some features such as clip deletion, emotion tags or generalization mechanisms.
        
        """
        assert isinstance(ECM, ECMs.Two_Layer) or isinstance(num_actions, int)
        
        if ECM is None:
            self.ECM = ECMs.Two_Layer(num_actions, glow, damp, softmax)
        else:
            self.ECM = ECM

        self.percept_processor = percept_processor        
        
        
    def get_action(self, 
                observation: object # data object passed from the environment/sensors
               )-> int : # The action to be performed.
        """
        Given a percept, returns an action. For basic PS, these processess are mainly executed by the ECM's deliberate function
        """
        percept = self.percept_processor(observation)
        action = self.ECM.deliberate(percept)
        return action

    def update(self, 
               reward #Value used for reinforcement. Likely a float
              ):
        """
        Given a reward, updates h-matrix. Updates g-matrix with glow.
        """
        self.ECM.learn(reward)

# Situated Agent

In [ ]:
#| export
class Situated_Agent(Abstract_Agent):
    def __init__(self, 
                 reflex_ECM = None, #if an ECM object is not given, a number of actions must be given with which to create one
                 episode_ECM = None, #if an ECM object is not given, a memory capacity must be given. This will be used to creat a new ECM
                 num_actions = None, # The number of available actions. If an ECM is not given, should be int
                 memory_capacity:int = None,
                 glow: float = 0.1, # The glow (or eta) parameter. Won't be used if ECM is given
                 damp: float = 0., # The damping (or gamma) parameter. Won't be used if ECM is given
                 reflex_softmax: float = 0.1, # The softmax (or beta) parameter. Won't be used if ECM is given
                 PS_softmax: float = 0.7,
                 focus: float = 0,
                 error_tolerance: float = 0.01,
                 min_expectation: float = 0.01,
                 deliberation_length: int = 1,
                 t = 0,
                 percept_processor = None, 
                 action_processor = None
                ):
        assert isinstance(reflex_ECM, ECMs.Priming_ECM) or isinstance(num_actions, int)
        if reflex_ECM is None:
            self.reflex_ECM = ECMs.Priming_ECM(num_actions, glow, damp, reflex_softmax)
        else:
            self.reflex_ECM = reflex_ECM

        assert isinstance(episode_ECM, ECMs.Episodic_Memory) or isinstance(memory_capacity, int)
        if episode_ECM is None:
            self.episode_ECM = Episodic_Memory(num_actions = self.reflex_ECM.num_actions, 
                                               capacity = memory_capacity, 
                                               softmax = PS_softmax,
                                               focus = focus,
                                               error_tolerance = error_tolerance,
                                               min_expectation = min_expectation,
                                               deliberation_length = deliberation_length,
                                               t = t
                                              )
        else:
            self.episode_ECM = episode_ECM
        assert self.reflex_ECM.num_actions == self.episode_ECM.num_actions
        self.num_actions = self.reflex_ECM.num_actions

        if percept_processor is None:
            self.percept_processor = perprocessors.action_factorizor(num_actions = self.num_actions)
        else:
            self.percept_processor = percept_processor

    def get_action(self, observation):
        action = self.reflex_ECM.deliberate(str(observation))
        percept = self.percept_processor.get_percept(observation, action)
        self.episode_ECM.deliberate(percept) #runs predictions, priming actions for next step.
        return(action)

    def update(self):
        reward = np.nanmean(self.episode_ECM.valences) - self.episode_ECM.surprise
        self.ECM.learn(reward)